## Numpy

Numpy is a convenient, Pythonic _toolkit_ for manipulating raw memory. It's primarily intended for data analysis applications:

In [ ]:
import numpy

array = numpy.array([0.0, 1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])
array[5:7]

In [ ]:
array[array > 3]

But it also lets you do bare metal stuff, like byte-swapping and casting.

In [ ]:
array.byteswap()

In [ ]:
array.view(dtype="int32")

A Numpy array object (`ndarray`) is essentially just a C array with a Python object attached. The Python object manages everything that is ordinarily unsafe about C arrays:

   * the length (to prevent overwrites)
   * the type (to prevent unintended casting), including unsigned integers
   * the byte order (ditto)
   * C vs Fortran order for multidimensional arrays (e.g. which index runs contiguously in memory: the first or last?)
   * whether this object owns the array or if it is just a "view".

Usually, when you create a new Numpy array (sometimes implicitly in an expression involving arrays), you want Numpy to allocate a new memory buffer and let the `ndarray` object own it. That is, when the `ndarray` is deleted, the buffer gets freed.

For performance, some Numpy operations give you a "view" into another array, rather than a copy:

In [ ]:
subarray = array[5:]
subarray

In [ ]:
subarray[2] = 999.99

In [ ]:
array

You can identify a "view" because it has a "base" reference to the array that it's viewing. By maintaining a reference, the view can ensure that the base doesn't get garbage collected until they're both out of scope.

In [ ]:
subarray.base is array

In [ ]:
array.base is None

But there's yet another case: sometimes you have a buffer already and want Numpy to wrap it. Maybe you want to use some of Numpy's vectorized functions on the data, or maybe you want to pass it to some software that only recognizes data in Numpy format (`<cough>` machine learning `<cough>`).

Anything that satisfies Python's "buffer" interface can become an `ndarray`.

In [ ]:
string = "hello there"
array = numpy.frombuffer(string, dtype=numpy.uint8)
array

In [ ]:
map(chr, array)

In [ ]:
array.base is string

With some effort, Numpy arrays can even wrap arbitrary regions of memory, given by an integer-valued pointer.

In [ ]:
import ctypes
libc = ctypes.cdll.LoadLibrary("libc.so.6")
libc.malloc.restype = ctypes.POINTER(ctypes.c_double)
ptr = libc.malloc(4096)
ptr

In [ ]:
ptr.__array_interface__ = {
    "version": 3,
    "typestr": numpy.ctypeslib._dtype(type(ptr.contents)).str,
    "data": (ctypes.addressof(ptr.contents), False),
    "shape": (4096,)
}
array = numpy.array(ptr, copy=False)
array

## Snake eating its tail again

Have you ever wondered what Python structs look like? You don't have to use the C API to delve into this. The `id(obj)` for some `obj` happens to be a numerical pointer to the object in memory. This fact is not guaranteed in future versions of Python (nor is it true in alternate implementations, such as Jython), but it's true for now.

In [ ]:
string = "hello there"
id(string)

In [ ]:
ptr = ctypes.cast(id(string), ctypes.POINTER(ctypes.c_uint8))
ptr.__array_interface__ = {
    "version": 3,
    "typestr": numpy.ctypeslib._dtype(type(ptr.contents)).str,
    "data": (ctypes.addressof(ptr.contents), False),
    "shape": (64,)
}
array = numpy.array(ptr, copy=False)
print map(chr, array)

Can you spot it?